# Table of Contents
 <p>

In [1]:
import os
import pandas as pd

import json
import pickle

In [ ]:
user = os.environ.get('MONGOD_DURAMAT_ADMIN_USER')
passwd = os.environ.get('MONGOD_DURAMAT_ADMIN_PASS')
leftover = os.environ.get('MONGOD_DURAMAT_LEFTOVER')
client = MongoClient('mongodb+srv://{}:{}@{}'.format(user, passwd, leftover))
client.database_names()

In [4]:
sys_meta = pd.read_pickle('sys_meta.pkl')

In [5]:
def read_time_series_data(dir_path='/Users/benellis/data_sets/PVoutput/Merged_Data/merged_csv'):
    dfs = {}
    mapper = {
        'SurfaceTemperatureCelsius': 't_surface (C)',
        'SurfaceDewpointTemperatureCelsius': 't_dewpoint (C)', 
        'SurfaceWetBulbTemperatureCelsius': 't_wetbulb (C)',
        'RelativeHumidityPercent': 'rh (%)', 
        'SurfaceAirPressureKilopascals': 'p_surface (kPa)',
        'CloudCoveragePercent': 'cloud_cover (%)', 
        'WindChillTemperatureCelsius': 't_wind_chill (C)',
        'ApparentTemperatureCelsius': 't_apparent (C)', 
        'WindSpeedKph': 'v_wind (kph)', 
        'WindDirectionDegrees': 'd_wind (deg)',
        'PrecipitationPreviousHourCentimeters': 'precip_prev_hr (cm)', 
        'DownwardSolarRadiationWsqm': 'downard_solar_irrad (Wm-2)',
        'DiffuseHorizontalRadiationWsqm': 'diffuse_horizontal_irrad (Wm-2)', 
        'DirectNormalIrradianceWsqm': 'direct_normal_irrad (Wm-2)',
        'MslPressureKilopascals': 'mslp (kPa)', 
        'HeatIndexCelsius': 'heat_index (C)', 
        'SnowfallCentimeters': 'snow (cm)',
        'SurfaceWindGustsKph': 'wind_gust (kph)', 
        'GlobalHorizontalIrradianceWsqm': 'global_horizontal_irrad (Wm-2)', 
        'poa_global': 'poa_global_irrad (Wm-2)',
        'poa_direct': 'poa_direct_irrad (Wm-2)', 
        'poa_diffuse': 'poa_diffuse_irrad (Wm-2)', 
        'poa_sky_diffuse': 'poa_diffuse_sky (Wm-2)', 
        'poa_ground_diffuse': 'poa_diffuse_ground (Wm-2)',
        'Power(W)': 'power (W)', 
        'Energy(Wh)': 'energy (Wh)'
    }

    for f in os.listdir(dir_path):
        sys_id = int(f.split('_')[0])
        df = pd.read_csv(os.path.join(dir_path, f))
        df.rename(columns=mapper, inplace=True)
        df.index = pd.to_datetime(df['local_time'])
        tz = sys_meta[sys_meta['ID'] == sys_id]
        tz = tz['timezone'].values[0]
        df.index = df.index.tz_localize(tz, ambiguous='infer') 
        # bsons[sys_id] = pickle.dumps(df)
        dfs[sys_id] = df
        
    sys_dfs = pd.DataFrame.from_dict(dfs, orient='index')
    sys_dfs['ID'] = sys_dfs.index
    sys_dfs = sys_dfs.rename(columns={0: 'time_series_data'})
    return sys_dfs


In [6]:
ts_df = read_time_series_data()

In [7]:
ts_df.head()

,time_series_data,ID
25601,local_time...,25601
35842,local_time...,35842
20485,local_time...,20485
33798,local_time...,33798
26632,local_time...,26632


In [8]:
ts_df.iloc[0]['time_series_data'].head()

,local_time,t_surface (C),t_dewpoint (C),t_wetbulb (C),rh (%),p_surface (kPa),cloud_cover (%),t_wind_chill (C),t_apparent (C),v_wind (kph),...,snow (cm),wind_gust (kph),global_horizontal_irrad (Wm-2),poa_global_irrad (Wm-2),poa_direct_irrad (Wm-2),poa_diffuse_irrad (Wm-2),poa_diffuse_sky (Wm-2),poa_diffuse_ground (Wm-2),power (W),energy (Wh)
local_time,,,,,,,,,,,,,,,,,,,,,
2014-03-03 00:00:00-08:00,2014-03-03 00:00:00,10.1,8.29,9.1,89,100.4,6,10.1,10.1,7.4,...,0.0,33.2,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,NaN
2014-03-03 01:00:00-08:00,2014-03-03 01:00:00,10.7,9.01,9.8,89,100.3,24,10.7,10.7,10.0,...,0.0,35.7,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,NaN
2014-03-03 02:00:00-08:00,2014-03-03 02:00:00,10.8,8.76,9.7,88,100.3,38,10.8,10.8,9.2,...,0.0,36.9,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,NaN
2014-03-03 03:00:00-08:00,2014-03-03 03:00:00,10.6,8.60,9.5,87,100.3,42,10.6,10.6,9.6,...,0.0,38.1,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,NaN
2014-03-03 04:00:00-08:00,2014-03-03 04:00:00,10.4,8.30,9.3,87,100.3,42,10.4,10.4,10.1,...,0.0,38.9,0.0,0.0,-0.0,0.0,0.0,0.0,NaN,NaN


In [48]:
import pvlib
import numpy as np
import matplotlib.pyplot as plt
import rdtools

%matplotlib inline
def rdtools_analysis(df, ts):
    # Specify the Metadata
    meta = {"latitude": df['latitude (deg)'],
            "longitude": df['longitude (deg)'],
            "timezone": df['timezone'],
            "tempco": None,
            "azimuth": df['orientation (deg)'],
            "tilt": df['array_tilt (deg)'],
            "pdc": df['system_size (W)'],
            "temp_model": 'open_rack_cell_polymerback'}
    
#     ts = pickle.loads(df['time_series_data'])
#     ts.index = pd.to_datetime(ts['local_time'])
    # TZ is required for irradiance transposition
#     ts.index = ts.index.tz_localize(meta['timezone'], ambiguous='infer') 

    # Chage power from kilowatts to watts
    # There is some missing data, but we can infer the frequency from the first several data points
    freq = pd.infer_freq(ts.index[:10])

    # And then set the frequency of the dataframe
    ts = ts.resample(freq).median()

    # Calculate energy yield in Wh
    ts['energy'] = ts['power (W)'] * pd.to_timedelta(ts['power (W)'].index.freq).total_seconds() / (3600.0)
    # ts['energy'] = ts['Energy(Wh)']

    # Calculate POA irradiance from DHI, GHI inputs
    loc = pvlib.location.Location(meta['latitude'], meta['longitude'], tz=meta['timezone'])
    sun = loc.get_solarposition(ts.index)

    # calculate the POA irradiance
    sky = pvlib.irradiance.isotropic(meta['tilt'], ts['diffuse_horizontal_irrad (Wm-2)'])
    ts['dni'] = (ts['global_horizontal_irrad (Wm-2)'] - ts['diffuse_horizontal_irrad (Wm-2)']) / np.cos(np.deg2rad(sun['zenith']))
    beam = pvlib.irradiance.beam_component(meta['tilt'], meta['azimuth'], sun['zenith'], sun['azimuth'], ts['direct_normal_irrad (Wm-2)'])
    ts['poa'] = beam + sky

    # Calculate cell temperature
    df_temp = pvlib.pvsystem.sapm_celltemp(ts['poa'], ts['v_wind (kph)'], ts['t_apparent (C)'], model=meta['temp_model'])
    ts['Tcell'] = df_temp['temp_cell']
    
    # Specify the keywords for the pvwatts model
    pvwatts_kws = {"poa_global" : ts['poa'],
                   "P_ref" : meta['pdc'],
                   "T_cell" : ts['Tcell'],
                   "G_ref" : 1000,
                   "T_ref": 25,
                   "gamma_pdc" : meta['tempco']}
    
    normalized, insolation = rdtools.normalize_with_pvwatts(ts['energy'], pvwatts_kws)

    ts['normalized'] = normalized
    ts['insolation'] = insolation

    nz_mask = (ts['normalized'] > 0)
    poa_mask = rdtools.poa_filter(ts['poa'])
    tcell_mask = rdtools.tcell_filter(ts['Tcell'])
    clip_mask = rdtools.clip_filter(ts['power (W)'])

    # filter the time series and keep only the columns needed for the
    # remaining steps
    filtered = ts[nz_mask & poa_mask & tcell_mask & clip_mask]
    filtered = filtered[['insolation', 'normalized']]
    daily = rdtools.aggregation_insol(filtered['normalized'], filtered['insolation'], frequency='D')
#     daily = rdtools.aggregation_insol(filtered['normalized'], filtered['insolation'], frequency='D')
       
#     fig, ax = plt.subplots()
#     daily.plot(ax=ax)

    # Calculate the degradation rate using the YoY method
    yoy_rd, yoy_ci, yoy_info = rdtools.degradation_year_on_year(daily)
    ols_rd, ols_ci, ols_info = rdtools.degradation_ols(daily)
    csd_rd, csd_ci, csd_info = rdtools.degradation_classical_decomposition(daily.fillna(method='ffill').fillna(method='bfill'))

#     print(yoy_rd, ols_rd, csd_rd)

    # Visualize the results
#     start = daily.index[0]
#     end = daily.index[-1]
#     years = (end - start).days / 365.0
#     yoy_values = yoy_info['YoY_values']

#     x = [start, end]
#     y = [1, 1 + (yoy_rd * years) / 100]

#     fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 3))
#     ax2.hist(yoy_values, label='YOY', bins=len(yoy_values)//40)
#     ax2.axvline(x=yoy_rd, color='black', linestyle='dashed', linewidth=3)
# #     ax2.set_xlim(-30,45)
#     ax2.annotate( u' $R_{d}$ = %.2f%%/yr \n confidence interval: \n %.2f to %.2f %%/yr' 
#                  %(yoy_rd, yoy_ci[0], yoy_ci[1]),  xy=(0.5, 0.7), xycoords='axes fraction',
#                 bbox=dict(facecolor='white', edgecolor=None, alpha = 0))
#     ax2.set_xlabel('Annual degradation (%)');

#     ax1.plot(daily.index, daily / yoy_info['renormalizing_factor'], 'o', alpha = 0.5)
#     ax1.plot(x, y, 'k--', linewidth=3)
#     ax1.set_xlabel('Date')
#     ax1.set_ylabel('Renormalized Energy')
# #     ax1.set_ylim(0.5, 1.1)
#     fig.autofmt_xdate()

    # fig.suptitle('Sensor-based degradation results');
    return daily, yoy_rd, ols_rd, csd_rd

In [71]:
def make_deg_df(meta_row, time_series):
    try:
        sys_id = meta_row['ID']
        daily, yoy, ols, csd = rdtools_analysis(meta_row, time_series)
        new_df = {'ID': sys_id, 'performance': pickle.dumps(daily), 'yoy_rd': yoy, 'ols_rd': ols, 'csd_rd': csd}
        return new_df
    except ValueError:
        return {'ID': sys_id, 'performance': pickle.dumps([]), 'yoy_rd': np.nan, 'ols_rd': np.nan, 'csd_rd': np.nan}

In [72]:
import concurrent.futures as cf

In [73]:
# def test(a, b):
#     return a + b

sys_meta = sys_meta.sort_values('ID')
ts_df = ts_df.sort_values('ID')

In [74]:
results = []
with cf.ProcessPoolExecutor() as ex:
    future_list = [ex.submit(make_deg_df, a, b['time_series_data']) 
                     for (idxa, a), (idxb, b) in zip(sys_meta.iterrows(), ts_df.iterrows())]
    for future in cf.as_completed(future_list):
        results.append(future.result())
# print(results)

In [76]:
len(pd.DataFrame(results).dropna())

409

In [77]:
rd_df = pd.DataFrame(results)

In [78]:
rd_df.head()

,ID,csd_rd,ols_rd,performance,yoy_rd
0,1519,2.318220,1.862156,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,-0.223590
1,1533,0.200015,4.007570,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,-1.346519
2,1095,4.364847,0.745874,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,0.839860
3,1421,-4.106687,0.099126,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,-1.072877
4,1384,0.220619,0.628772,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,0.379126


In [79]:
(rd_df.memory_usage(deep=True) * 1e-6).sum()

14.178514999999999

In [80]:
sys_meta_rd = sys_meta.merge(rd_df, left_on='ID', right_on='ID')

In [81]:
sys_meta_rd.head()

,ID,system_name,system_size (W),postcode,n_panels,p_panel (W),panel_brand,n_inverters,p_inverter (W),inverter_brand,...,elevation (m),orientation (deg),climate,county,state (abbr),state (name),csd_rd,ols_rd,performance,yoy_rd
0,1095,FewsFord,2160,27712,12,180,Day4,1,6000,Xantrex xw6048,...,178.627951,180.0,Cfa,Orange,NC,North Carolina,4.364847,0.745874,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,0.839860
1,1384,Eric's System,2760,55116,12,230,Siliken SLK60P6L,12,190,Enphase M190,...,290.865427,180.0,Dfb,Ramsey,MN,Minnesota,0.220619,0.628772,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,0.379126
2,1421,Bartfamily PV Array,3360,21009,16,210,Evergreen ES-A-210-fa2,1,3000,SMA Sunnyboy 3000US,...,51.222543,180.0,Cfa,Harford,MD,Maryland,-4.106687,0.099126,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,-1.072877
3,1519,Ohiobiker,9870,43138,42,235,NU-U235F1,42,250,Enphase Energy M250,...,296.185038,135.0,Cfa,Hocking,OH,Ohio,2.318220,1.862156,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,-0.223590
4,1533,University Place WA Array,16660,98466,68,245,Sharp 175 & LG 270,68,210,Enphase,...,105.450580,180.0,Csb,Pierce,WA,Washington,0.200015,4.007570,b'\x80\x03cpandas.core.series\nSeries\nq\x00)\...,-1.346519


In [85]:
(sys_meta_rd.memory_usage(deep=True) * 1e-6).sum()

14.949375999999999

In [87]:
from pymongo import MongoClient
user = os.environ.get('MONGOD_DURAMAT_ADMIN_USER')
passwd = os.environ.get('MONGOD_DURAMAT_ADMIN_PASS')
leftover = os.environ.get('MONGOD_DURAMAT_LEFTOVER')
client = MongoClient('mongodb+srv://{}:{}@{}'.format(user, passwd, leftover))
client.database_names()

['admin', 'local', 'pvdaq', 'pvdata', 'pvproduction', 'test_db']

In [88]:
client.drop_database('pvdata')

In [89]:
client.database_names()

['admin', 'local', 'pvdaq', 'pvproduction', 'test_db']

In [90]:
newdb = client['pvdata']

In [91]:
newcoll = newdb['appdata']

In [104]:
print(sys_meta_rd.to_dict('records')[0]['ID'])
pickle.loads(sys_meta_rd.to_dict('records')[0]['performance'])

1095


local_time
2011-08-05 00:00:00-04:00    0.182685
2011-08-06 00:00:00-04:00    0.766664
2011-08-07 00:00:00-04:00    0.502401
2011-08-08 00:00:00-04:00    0.520899
2011-08-09 00:00:00-04:00    0.574375
2011-08-10 00:00:00-04:00    0.404636
2011-08-11 00:00:00-04:00    0.465352
2011-08-12 00:00:00-04:00    0.592225
2011-08-13 00:00:00-04:00    0.337684
2011-08-14 00:00:00-04:00    0.453886
2011-08-15 00:00:00-04:00    0.536029
2011-08-16 00:00:00-04:00    0.489514
2011-08-17 00:00:00-04:00    0.393527
2011-08-18 00:00:00-04:00    0.617929
2011-08-19 00:00:00-04:00    0.537656
2011-08-20 00:00:00-04:00    0.537975
2011-08-21 00:00:00-04:00    0.596172
2011-08-22 00:00:00-04:00    0.544453
2011-08-23 00:00:00-04:00    0.441483
2011-08-24 00:00:00-04:00    0.462724
2011-08-25 00:00:00-04:00    0.525880
2011-08-26 00:00:00-04:00    0.465854
2011-08-27 00:00:00-04:00         NaN
2011-08-28 00:00:00-04:00    0.534384
2011-08-29 00:00:00-04:00    0.758997
2011-08-30 00:00:00-04:00    0.525268
2

In [99]:
sys_meta_rd.to_pickle('sys_meta_rd.pkl')

In [100]:
sys_meta_rd.where(sys_meta_rd['system_name'] == 'FewsFord').dropna().keys()

Index(['ID', 'system_name', 'system_size (W)', 'postcode', 'n_panels',
       'p_panel (W)', 'panel_brand', 'n_inverters', 'p_inverter (W)',
       'inverter_brand', 'orientation (cardinal)', 'array_tilt (deg)', 'shade',
       'install_date', 'active_days', 'latitude (deg)', 'longitude (deg)',
       'elevation (ft)', 'status_interval (min)', 'WS_1', 'WS_2', 'WS_3',
       'WS_4', 'WS_5', 'timezone', 'WS_1 Data %', 'WS_2 Data %', 'WS_3 Data %',
       'WS_4 Data %', 'WS_5 Data %', 'Most Data WS', 'last_update',
       'Most Data WS Name', 'DB_ID', 'elevation (m)', 'orientation (deg)',
       'climate', 'county', 'state (abbr)', 'state (name)', 'csd_rd', 'ols_rd',
       'performance', 'yoy_rd'],
      dtype='object')

In [94]:
newcoll.insert_many(sys_meta_rd.to_dict('records'))

In [96]:
list(newcoll.find({'ID': 1095}, {'ID': 1, 'system_name': 1, 'system_size (W)': 1}))

[{'ID': 1095,
  '_id': ObjectId('5b197800bd49aab35deb1cfb'),
  'system_name': 'FewsFord',
  'system_size (W)': 2160}]

In [106]:
pd.cut(sys_meta_rd['system_size (W)'], 10).astype('str')

0       (72.685, 11416.5]
1       (72.685, 11416.5]
2       (72.685, 11416.5]
3       (72.685, 11416.5]
4      (11416.5, 22648.0]
5       (72.685, 11416.5]
6       (72.685, 11416.5]
7       (72.685, 11416.5]
8       (72.685, 11416.5]
9       (72.685, 11416.5]
10      (72.685, 11416.5]
11     (11416.5, 22648.0]
12      (72.685, 11416.5]
13      (72.685, 11416.5]
14      (72.685, 11416.5]
15      (72.685, 11416.5]
16      (72.685, 11416.5]
17      (72.685, 11416.5]
18      (72.685, 11416.5]
19      (72.685, 11416.5]
20      (72.685, 11416.5]
21      (72.685, 11416.5]
22      (72.685, 11416.5]
23      (72.685, 11416.5]
24      (72.685, 11416.5]
25     (11416.5, 22648.0]
26      (72.685, 11416.5]
27     (11416.5, 22648.0]
28      (72.685, 11416.5]
29      (72.685, 11416.5]
              ...        
543     (72.685, 11416.5]
544     (72.685, 11416.5]
545    (11416.5, 22648.0]
546     (72.685, 11416.5]
547     (72.685, 11416.5]
548     (72.685, 11416.5]
549    (33879.5, 45111.0]
550     (72.

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

In [ ]:
traces = []
min_p = 1e6
max_p = -1e6
for idx, i in full_df.iterrows():
    ts = pickle.loads(i['time_series_data'])
    ts = ts[ts['power (W)'] > 0]
    ts = ts.resample('M').median()
    ts = ts.dropna(subset=['power (W)'])
    ts['power (W)'] = ts['power (W)'] / i['system_size (W)']
    min_p = min(min_p, ts['power (W)'].min())
    max_p = max(max_p, ts['power (W)'].max())
    if idx == 10:
        showscale = True
    else:
        showscale = False
    traces.append(
        go.Scatter(x=ts.index, y=['[{}]'.format(i['ID'])] * len(ts),
                   mode='markers', name=i['ID'],
                   marker={'color': ts['power (W)'], 'symbol': 'square', 
                           'showscale': showscale, 'cmin': min_p, 'cmax': max_p, 'colorscale': 'RdBu',
                          'colorbar': {'title': 'W/Wp', 'titleside': 'right'}},
                  hoverinfo='text', text=['{}: {:.3f}'.format(i['ID'], x) for x in ts['power (W)']]) 
    )
    if idx == 100: break
layout = go.Layout(legend=dict(orientation='h'))
fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [ ]:
traces = []
min_p = 1e6
max_p = -1e6
vals = {}
for idx, i in full_df.iterrows():
    ts = pickle.loads(i['time_series_data'])
    ts = ts[ts['power (W)'] > 0]
    ts = ts.resample('M').mean()
    ts.index = [i.date() for i in ts.index]
    ts['power (W)'] = ts['power (W)'] / i['system_size (W)']
    vals[i['ID']] = ts['power (W)']
    if idx == 300: break
        
df2 = pd.DataFrame.from_dict(vals, orient='index')
df2 = df2.T
df2 = df2.sort_index()

df2

traces = [go.Heatmap(x=df2.index, y=['[{}]'.format(i) for i in df2.keys()], z=df2.values.T, colorbar={'title': 'W/Wp', 'titleside': 'right'})]
# layout = go.Layout(height=500, width=(500 / len(df2.keys())) * len(df2.index))
layout = go.Layout(width=600, xaxis={'title': 'Date'}, yaxis={'title': 'System ID'})
fig = go.Figure(data=traces, layout=layout)
iplot(fig)